# Challenge: Promotions

In this challenge, you'll develop codes to parse and analyze data returned from another API on Zalando such as [Promos homme (Men's Promotions)
](https://www.zalando.fr/promo-homme/) or [Promos femme (Women's Promotions)](https://www.zalando.fr/promo-femme/). The workflow is almost the same as in the guided lesson but you'll work with different data.

## Obtaining the link

Wrote your codes in the cell below to obtain the data from the API endpoint you choose. A recap of the workflow:

1. Examine the webpages and choose one that you want to work with.

1. Use Google Chrome's DevTools to inspect the XHR network requests. Find out the API endpoint that serves data to the webpage.

1. Test the API endpoint in the browser to verify its data.

1. Change the page number offset of the API URL to test if it's working.

In [2]:
# your code here
url='https://www.zalando.fr/api/catalog/articles?categories=homme&limit=84&offset=84&sort=sale'

## Reading the data

In the next cell, use Python to obtain data from the API endpoint you chose in the previous step. Workflow:

1. Import libraries.

1. Define the initial API endpoint URL.

1. Make request to obtain data of the 1st page. Flatten the data and store it in an empty object variable.

1. Find out the total page count in the 1st page data.

1. Use a FOR loop to make requests for the additional pages from 2 to page count. Append the data of each additional page to the flatterned data object.

1. Print and review the data you obtained.

In [12]:
# your code here
import re
import json
import requests
import urllib.request
import pandas as pd

request = urllib.request.urlopen(url)
data = json.load(request)
data.keys()
pages=data['pagination']['page_count']
print(pages)

HTTPError: HTTP Error 500: Internal Server Error

In [11]:
df=pd.DataFrame()
for i in range(2):
    k=84*i
    url='https://www.zalando.fr/api/catalog/articles?categories=promo-femme&homme=84&offset=84%s&sort=sale'
    request = urllib.request.urlopen(url % k)
    data = json.load(request)
    flat_data = pd.json_normalize(data)
    flat_data1 = pd.json_normalize(flat_data.articles[0])
    flat_data1=flat_data1.set_index('sku')
    df = df.append(flat_data1)
df

,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,delivery_promises,price.original,price.promotional,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only
sku,,,,,,,,,,,,,,,,
AD121A0ER-J11,Legging - power pink/white,"[30, 32, 34, 36, 38, 40, 42, 44, 46, 48]",adidas-originals-tight-legging-ad121a0er-j11,[{'path': 'spp-media-p1/0d6ef578176130cbad4928...,adidas Originals,False,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -10...",clothing,[],"29,95 €","26,95 €",True,False,True,False
JY121I0BY-C11,JDYGADOT - Pullover - light grey melange,"[XS, S, M, L, XL]",jdy-jdygadot-pullover-light-grey-melange-jy121...,[{'path': 'spp-media-p1/c32182e9d1c8390abec236...,JDY,False,[],"[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,[],"19,99 €","15,99 €",False,False,False,False
NAA21N00S-K11,HIGH WAIST - Jeans Skinny - mid blue,"[34, 36, 38, 40, 42, 44]",na-kd-high-waist-jeans-skinny-mid-blue-naa21n0...,[{'path': 'spp-media-p1/feb9e6e839dd35ef9b1564...,NA-KD,False,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -54...",clothing,[],"20,95 €","16,76 €",True,False,True,False
ON321I0YB-I11,ONLCAVIAR - Pullover - wild ginger,"[XS, S, M, L, XL]",only-onlcaviar-pullover-on321i0yb-i11,[{'path': 'spp-media-p1/679401163a633594b9c030...,ONLY,False,[],"[{'key': 'discountRate', 'value': '-10%', 'tra...",clothing,[],"19,99 €","17,99 €",False,False,False,False
NI121A09O-N11,Pantalon de survêtement - olive flak/white,"[XS, S, M, L, XL, XXL]",nike-sportswear-w-nsw-essntl-pant-reg-flc-pant...,[{'path': 'spp-media-p1/1c2158c7f4ee3bf3a263ec...,Nike Sportswear,False,[],"[{'key': 'discountRate', 'value': '-10%', 'tra...",clothing,[],"38,95 €","35,05 €",False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GU151H2CS-G11,Sac bandoulière - red,[One Size],guess-becca-double-zip-crossbody-sac-bandoulie...,[{'path': 'spp-media-p1/b65c871c7a233897b7212f...,Guess,False,[],"[{'key': 'discountRate', 'value': '-25%', 'tra...",accessoires,[],"99,95 €","74,95 €",False,False,False,False
DE151Q02K-Q11,ALKALINA POSITANO - Sac à dos - black,[One Size],desigual-alkalina-positano-sac-a-dos-black-de1...,[{'path': 'spp-media-p1/b0a369617cce358386a765...,Desigual,False,[],"[{'key': 'discountRate', 'value': '-25%', 'tra...",accessoires,[],"79,95 €","59,95 €",False,False,False,False
RAD11B03K-O11,BELLA - Escarpins à talons hauts - brown,"[36, 37, 38, 39, 40, 41]",raid-bella-escarpins-brown-rad11b03k-o11,[{'path': 'spp-media-p1/fc04938a3b42393fb4d08c...,RAID,False,[],"[{'key': 'discountRate', 'value': '-50%', 'tra...",shoe,[],"40,99 €","20,49 €",False,False,False,False


## Bonus

Extract the following information from the data:

* The trending brand.

* The product(s) with the highest discount.

* The sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices).

In [ ]:
# your code here